In [1]:
import numpy as np
import pandas as pd
import itertools
import math
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import date, timedelta
plt.rcParams['figure.figsize'] = [15, 10]
import plotly.express as px
import plotly.graph_objects as go

The Function Closing price helps to retrieve data using the yahoo finance library. You just have to enter the ticker and how many years of data you need and will return the dataframe with the information. 

In [2]:
def closing_price(ticker,years):
    Start = date.today() - timedelta(365 *years)
    Start.strftime('%Y-%m-%d')
    End = date.today() + timedelta(2)
    End.strftime('%Y-%m-%d')
    Asset = pd.DataFrame(
        yf.download(ticker, start=Start, end=End)['Adj Close'])
    return Asset

The following used the rolling property of the dat frame to roll over the data in a window of **n** elements. The **.mean** at the end is the operation we want to be made with the mini array of elements that takes the rolling function. 

Is like telling to python to divide the data in **len(data)-n** arrays and with each one of them you will obtain the mean, and the result is going to be a new array, which is the result you give as a dataframe.

In [3]:
def sma(data, n):
    sma = data.rolling(window=n).mean()
    return pd.DataFrame(sma)

This function takes tree dataframes. One, with the original data, and two with different sma dataframes. One should be shorter than the other. We are going to compare the values within them and take note of their relation. In this function, we define the strartegy to use. We have three main rules:
* If the short element is grates than the long element: we buy.
* If the long element is grater than the short element: we sell.
* If they are equal, or the default action: we keep with the previous strategy

Along the analysis of the data, we are going to take a "note" with a flag named signal when should buy of cell. this information is then returned.

In [6]:
def sma_strategy(data, short_window, long_window):
    sma1 = short_window
    sma2 = long_window
    buy_price = []
    sell_price = []
    sma_signal = []
    signal = 0

    for i in range(len(data)):
        if sma1.values[i] > sma2.values[i]:
            if signal != 1:
                buy_price.append(data.values[i][0])
                sell_price.append(np.nan)
                signal = 1
                sma_signal.append(signal)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                sma_signal.append(0)
        elif sma2.values[i] > sma1.values[i]:
            if signal != -1:
                buy_price.append(np.nan)
                sell_price.append(data.values[i][0])
                signal = -1
                sma_signal.append(-1)
            else:
                buy_price.append(np.nan)
                sell_price.append(np.nan)
                sma_signal.append(0)
        else:
            buy_price.append(np.nan)
            sell_price.append(np.nan)
            sma_signal.append(0)

    return buy_price, sell_price, sma_signal


SMA is a way of smooth the original time serie. while increasing the window used in the calculation of SMA, we flaten the imperfections of the curve. This is great for inspecting the tendency of the curve or general behaviours. 

In [7]:
TESLA = closing_price('TSLA',1)
plots = dict()
for window in range(1, 30, 5):
    aux = sma(TESLA, window)
    l = "w " + str(window)
    plots[l] = list(itertools.chain.from_iterable(aux.values))
df = pd.DataFrame.from_dict(plots)
fig = px.line(df, title='Simple moving averge for several windows')
fig.show()

[*********************100%***********************]  1 of 1 completed


# SMA Strategy

In [13]:
long_window = 20
short_window = 50
TESLA = closing_price('TSLA',3)
TESLA_SMA_S = sma(TESLA, short_window)
TESLA_SMA_L = sma(TESLA, long_window)
buy, sell, signals = sma_strategy(TESLA, TESLA_SMA_S, TESLA_SMA_L)
### lets create a dataframe with the values we have and the shape we want
strategy = pd.DataFrame(columns=[
    'date',
    'price',
    'signal',
])
for i in range(0, len(TESLA)):
    if not math.isnan(buy[i]):
        signal = 'buy'
    elif not math.isnan(sell[i]):
        signal = 'sell'
    else:
        signal = False
    strategy.loc[i] = [
        TESLA['Adj Close'].index[i], TESLA['Adj Close'].values[i], signal
    ]


[*********************100%***********************]  1 of 1 completed


C:\Users\RogelioBasurto\AppData\Local\Temp\ipykernel_23004\62383017.py:10: RuntimeWarning:

invalid value encountered in greater

C:\Users\RogelioBasurto\AppData\Local\Temp\ipykernel_23004\62383017.py:20: RuntimeWarning:

invalid value encountered in greater



In [14]:
fig = fig = go.Figure()
fig.add_trace(go.Scatter(x=TESLA['Adj Close'].index, y=TESLA['Adj Close'].values,name="Closing value"))
fig.add_trace(go.Scatter(x=TESLA_SMA_S['Adj Close'].index, y=TESLA_SMA_S['Adj Close'].values,name="SMA 1"))
fig.add_trace(go.Scatter(x=TESLA_SMA_L['Adj Close'].index, y=TESLA_SMA_L['Adj Close'].values,name="SMA 2"))

fig.add_trace(go.Scatter(x=TESLA_SMA_L['Adj Close'].index, y=buy,name="buy",mode='markers',marker_size=15,marker_symbol ='triangle-up'))
fig.add_trace(go.Scatter(x=TESLA_SMA_L['Adj Close'].index, y=sell,name="sell",mode='markers',marker_size=15,marker_symbol ='triangle-down'))
fig.show()

# Backtesting

In [15]:
def backtesting(strategy, investment):
    purchase = 0
    stash = investment
    for i in range(0, len(strategy)):
        if strategy.loc[i]['signal']:
            if strategy.loc[i]['signal'] == 'buy':
                purchase = stash / strategy.loc[i]['price']
                stash = 0
            if strategy.loc[i]['signal'] == 'sell':
                stash = stash + strategy.loc[i]['price'] * purchase
                purchase = 0

    return stash - investment

In [16]:
investment = 1000
profit = backtesting(strategy,investment)
profit

2587.303365222583

# Referencias
*  IIFL Securities. (n.d.). What is a Simple Moving Average Trading Strategy? IIFL Knowledge Center. Retrieved August 14, 2022, from https://www.indiainfoline.com/knowledge-center/trading-account/what-is-a-simple-moving-average-trading-strategy#:~:text=Simple%20Moving%20Average%20(SMA)%3A,as%20the%20average%20price%20changes. 
*  IIFL Securities. (n.d.). What is a Simple Moving Average Trading Strategy? IIFL Knowledge Center. Retrieved August 14, 2022, from https://www.indiainfoline.com/knowledge-center/trading-account/what-is-a-simple-moving-average-trading-strategy#:~:text=Simple%20Moving%20Average%20(SMA)%3A,as%20the%20average%20price%20changes. 
* Analyst, A. G. E. Q., &amp; Georgios Efstathopoulos Quantitative AnalystPhD in Applied Mathematics and Statistics. Analyst working on quantitative trading. (n.d.). Python for Finance, part 3: Moving average trading strategy. Learn Data Science - Tutorials, Books, Courses, and More. Retrieved August 14, 2022, from https://www.learndatasci.com/tutorials/python-finance-part-3-moving-average-trading-strategy/ 